In [1]:
import pandas as pd

import re
import string
from kcontractions import kkcontractions
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df=pd.read_csv('mydataset.csv')

In [5]:
df.head()

,news,type
0,Ad sales boost Time Warner profit\n \n Quarter...,business
1,Dollar gains on Greenspan speech\n \n The doll...,business
2,Yukos unit buyer faces loan claim\n \n The own...,business
3,High fuel prices hit BA's profits\n \n British...,business
4,Pernod takeover talk lifts Domecq\n \n Shares ...,business


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3121 entries, 0 to 3120
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   news    3121 non-null   object
 1   type    3121 non-null   object
dtypes: object(2)
memory usage: 48.9+ KB


labelling the type

In [8]:
df['type_id']=df.type.map({
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
})

In [9]:
df.head()

,news,type,type_id
0,Ad sales boost Time Warner profit\n \n Quarter...,business,0
1,Dollar gains on Greenspan speech\n \n The doll...,business,0
2,Yukos unit buyer faces loan claim\n \n The own...,business,0
3,High fuel prices hit BA's profits\n \n British...,business,0
4,Pernod takeover talk lifts Domecq\n \n Shares ...,business,0


In [11]:
def remove_htmls(text):
    pattern=re.compile('<.*?>')
    return pattern.sub(r'',text)

In [12]:
def remove_urls(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [13]:
wlm=WordNetLemmatizer()

In [14]:
def transformed_text(text):
    text=text.lower()
    text=remove_htmls(text)
    text=remove_urls(text)
    text=nltk.word_tokenize(text)
    
    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        y.append(wlm.lemmatize(i))
    return " ".join(y)
        

In [15]:
df['news']=df['news'].apply(transformed_text)

In [16]:
df.head()

,news,type,type_id
0,ad sale boost time warner profit quarterly pro...,business,0
1,dollar gain greenspan speech dollar hit highes...,business,0
2,yukos unit buyer face loan claim owner embattl...,business,0
3,high fuel price hit ba profit british airway b...,business,0
4,pernod takeover talk lift domecq share uk drin...,business,0


In [17]:
df['news'][26]

'steel firm cut job mittal steel one world largest steel producer could cut job next five year chief executive said company due complete acquisition u firm isg next month making one largest global firm kind however lakshmi mittal told investor combined company shed thousand job magnate say job loss would fall mr mittal told u investor acquisition international steel group completed company would aim reduce workforce annually could see workforce trimmed staff 2010 investing modernisation employee go mr mittal told conference chicago mittal steel formed last year mr mittal lnm holding merged dutch firm ispat combination mittal steel isg would annual sale 32bn euro production capacity 70 million tonne mittal steel spokeman said decision job cut made yet trying create sustainable steel industry want invest new technology spokesman said mittal steel operation 14 country many business particularly eastern europe previously state owned huge workforce employ staff kazakhstan alone large operat

In [18]:
tfidf=TfidfVectorizer()

In [19]:
X=tfidf.fit_transform(df['news']).toarray()

In [20]:
X.shape

(3121, 25278)

In [21]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
Y=df['type_id'].values

In [24]:
Y.shape

(3121,)

In [25]:
X_train, X_test, y_train, y_test =train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=2022,
    stratify=Y
)

In [26]:
nbCls=MultinomialNB()

In [27]:
nbCls.fit(X_train,y_train)
y_pred1=nbCls.predict(X_test)

In [28]:
print(classification_report(y_test, y_pred1))
print('Confusion Matrix :')
print("Confusion Matrix", confusion_matrix(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90       204
           1       0.94      0.99      0.97       155
           2       1.00      0.64      0.78        84
           3       0.99      0.98      0.99       102
           4       1.00      0.71      0.83        80

    accuracy                           0.91       625
   macro avg       0.95      0.86      0.89       625
weighted avg       0.92      0.91      0.90       625

Confusion Matrix :
Confusion Matrix [[204   0   0   0   0]
 [  2 153   0   0   0]
 [ 30   0  54   0   0]
 [  1   1   0 100   0]
 [ 14   8   0   1  57]]


In [29]:
import pickle
pickle.dump(tfidf,open('vectorizednb.pkl','wb'))
pickle.dump(nbCls,open('modelnb.pkl','wb'))